# 0. Imports

In [2]:

import os
import math
import json
import skopt
import talib
import pickle
import dateutil.relativedelta

import numpy as np
import pandas as pd
import datetime as dt
import xgboost as xgb
import vectorbt as vbt

from histDataHandler import loadSuchData
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from binance_historical_data import BinanceDataDumper
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error


## 0.1 Model name, timeperiods

In [3]:

modelName = "articleModel"

rollingMeanWindow = 100
predictionHorizon = rollingMeanWindow+1

timePeriods = [
    3,
    5,
    10,
    25,
    3*24,
    7*24,
    15*24,
    20*24,
    30*24,
    45*24,
    3*30*24,
    6*30*24,
    ]

frequency="1h"

modelParamsDict = {
    "timePeriods":timePeriods,
    "rollingMeanWindow":rollingMeanWindow,
    "predictionHorizon":predictionHorizon,
    "frequency":frequency,
}


# 1. Load data

In [4]:

initialDate = "2017-01-01"

data = vbt.BinanceData.download(
    "BTCUSDT",
    start = initialDate,
    interval = frequency).get(["Open", "High", "Low", "Close", "Volume"])

data


0it [00:00, ?it/s]

,Open,High,Low,Close,Volume
Open time,,,,,
2017-08-17 04:00:00+00:00,4261.48,4313.62,4261.32,4308.83,47.181009
2017-08-17 05:00:00+00:00,4308.83,4328.69,4291.37,4315.32,23.234916
2017-08-17 06:00:00+00:00,4330.29,4345.45,4309.37,4324.35,7.229691
2017-08-17 07:00:00+00:00,4316.62,4349.99,4287.41,4349.99,4.443249
2017-08-17 08:00:00+00:00,4333.32,4377.85,4333.32,4360.69,0.972807
...,...,...,...,...,...
2023-04-20 18:00:00+00:00,28452.21,28526.93,28280.96,28360.49,4310.026890
2023-04-20 19:00:00+00:00,28360.48,28361.83,28010.00,28104.38,7082.277310
2023-04-20 20:00:00+00:00,28104.38,28321.69,28012.00,28204.56,3169.979330


# 2. Split data

Splitting data functions to be used later in training

In [4]:

def splitData(
    TrainSize,
    ValSize,
    TestSize,
    blockLengthInDays,
    data):
    """
    Split data into Train, Val and Test sets.
    The data is split into blocks of blockLengthInDays days.
    The data is then split into Train, Val and Test sets.
    """
    totSize = TrainSize+ValSize+TestSize
    TrainRatio = TrainSize/totSize
    ValRatio = ValSize/totSize
    # Test ratio is implicit from totSize

    datetimeIndex = data.index

    startTimeStamp = datetimeIndex[0]
    endTimeStamp = datetimeIndex[-1]
    blockLength = dt.timedelta(days=blockLengthInDays)

    nBlocks = math.floor((endTimeStamp-startTimeStamp)/blockLength)

    Train = pd.DataFrame()
    Val = pd.DataFrame()
    Test = pd.DataFrame()

    for i in range(nBlocks):
        TrainStart = endTimeStamp-(i+1)*blockLength
        TrainEndValStart = TrainStart + TrainRatio*blockLength
        ValEndTestStart = TrainEndValStart + ValRatio*blockLength
        TestEnd = endTimeStamp-(i)*blockLength

        Train = pd.concat([Train,data[TrainStart:TrainEndValStart]])
        Val = pd.concat([Val,data[TrainEndValStart:ValEndTestStart]])
        Test = pd.concat([Test,data[ValEndTestStart:TestEnd]])

    return Train, Val, Test

def splitDataRandom(
    TrainSize,
    ValSize,
    TestSize,
    data):
    """
    Split data into Train, Val and Test sets.
    Each sample in Train, Val and Test are randomly picked from data
    1- randomly pick a sample from data
    2- roll a random number between 0 and totSize
    3- if the number is smaller than TrainSize, add the sample to Train, else if it is smaller than TrainSize+ValSize, add the sample to Val, else add the sample to Test
    4- remove the sample from data so it doesn't get picked again
    """
    totSize = TrainSize+ValSize+TestSize
    trainCutOff = TrainSize/totSize
    valCutOff = (TrainSize+ValSize)/totSize

    # initialize Train, Val and Test
    TrainSel = []
    ValSel = []
    TestSel = []

    dataLen = len(data)

    #initialize loop
    for i in range(dataLen):
        # roll random float between 0 and 1
        randomRoll = np.random.rand()

        # add sample to Train, Val or Test
        if randomRoll < trainCutOff:
            TrainSel.append(i)
        elif randomRoll < valCutOff:
            ValSel.append(i)
        else:
            TestSel.append(i)
    
    Train = data.iloc[TrainSel]
    Val = data.iloc[ValSel]
    Test = data.iloc[TestSel]

    return Train, Val, Test


# 3. Generate target

Generate the target the model will be trained to predict

Currently the model will be trained to predict the percentual change in a determined Moving Average (MA)

The format is:
    
        - 1%  Up prediction -> 101
    
        - 10% Up prediction -> 110

In [5]:

def genTarget(data, rollingMeanWindow, predictionHorizon):
    """
    Generate target column for prediction.
    The target column is the rolling mean of the Close price.
    The rolling mean is shifted by predictionHorizon.
    """
    currentMean = talib.MA(data["Close"],timeperiod=rollingMeanWindow)
    futureMean = talib.MA(data["Close"],timeperiod=rollingMeanWindow).shift(-predictionHorizon)
    # compute percentual difference between current and future mean
    target = 100+((futureMean-currentMean)*100/currentMean)
    return pd.concat([data,target.rename("Target")],axis=1)


# 4. Feature engineering

### 4.1 Date features

Process data index to generate datetime features


In [6]:

def genDate(data):
    """
    Generate date columns from index.
    """
    data["month"] = data.index.month
    data["day"] = data.index.day
    data["dayofweek"] = data.index.dayofweek
    data["hour"] = data.index.hour
    return data


### 4.2 Process Open High Low


In [7]:
def processOHL(data):
    """
    Generate OHLV.
    """
    newCols = pd.DataFrame()
    data["Open"] = data[["Open","Close"]].apply(lambda x: (x["Open"]-x["Close"])/x["Close"], axis=1)
    data["High"] = data[["High","Close"]].apply(lambda x: (x["High"]-x["Close"])/x["Close"], axis=1)
    data["Low"] = data[["Low","Close"]].apply(lambda x: (x["Low"]-x["Close"])/x["Close"], axis=1)
    
    return data

### 4.3 Technical indicators

Some sample indicators from talib

Edit this session to add or remove indicators to model

In [9]:

def genBollingerBand(data, timeperiods, colname):
    """
    Generate Bollinger Bands.
    """
    newCols = pd.DataFrame()
    for timeperiod in timeperiods:
        newCols["{}_upper_band_{}m".format(colname, timeperiod)], newCols["{}_middle_band_{}m".format(colname, timeperiod)], newCols["{}_lower_band_{}m".format(colname, timeperiod)] = talib.BBANDS(data[colname], timeperiod=timeperiod)
    return newCols

def genRSI(data, timeperiods, colname):
    """
    Generate RSI.
    """
    newCols = pd.DataFrame()
    for timeperiod in timeperiods:
        newCols["{}_rsi_{}m".format(colname, timeperiod)] = talib.RSI(data[colname], timeperiod=timeperiod)
    return newCols

def genPercentChange(data, timeperiods, colname):
    """
    Generate percent change.
    """
    newCols = pd.DataFrame()
    for timeperiod in timeperiods:
        newCols["{}_percent_change_{}m".format(colname, timeperiod)] = data[colname].pct_change(timeperiod)
    return newCols

def genATR(data, timeperiods):
    """
    Generate ATR.
    """
    newCols = pd.DataFrame()
    for timeperiod in timeperiods:
        newCols["atr_{}m".format(timeperiod)] = talib.ATR(data["High"], data["Low"], data["Close"], timeperiod=timeperiod)
    return newCols



In [10]:

def genTechnicalIndicators(data, timeperiods):
       """
       Generate technical indicators.
       """

       # Close
       data = pd.concat([data, genBollingerBand(data, timeperiods=timeperiods, colname="Close")], axis=1)
       data = pd.concat([data, genRSI(data, timeperiods=timeperiods, colname="Close")], axis=1)
       data = pd.concat([data, genPercentChange(data, timeperiods=timeperiods, colname="Close")], axis=1)
       data = pd.concat([data, genATR(data, timeperiods=timeperiods)], axis=1)

       #Volume
       data = pd.concat([data, genBollingerBand(data, timeperiods=timeperiods, colname="Volume")], axis=1)
       data = pd.concat([data, genRSI(data, timeperiods=timeperiods, colname="Volume")], axis=1)
       data = pd.concat([data, genPercentChange(data, timeperiods=timeperiods, colname="Volume")], axis=1)

       #OBV
       data["OBV"] = talib.OBV(data["Close"],data["Volume"])

       # drop all na and infinite values
       data = data.replace([np.inf, -np.inf], np.nan)
       data = data.dropna()

       return data


# 5. Data preparation

## 5.1 Rescaling

Rescaling functions

We are using only MinMaxScaler for now

Cyclical features will go through a different process

In [11]:

def rescale_gen(data,modelName,save=False):
    """
    Rescale data using MinMaxScaler and save scaler to model folder
    """
    mms = MinMaxScaler()
    dir = f"../models/{modelName}/scalers/"
    # if dir doesnt exist, create it
    if not os.path.exists(dir):
        os.makedirs(dir)

    for i in data.columns:
        # do not scale cyclical features
        # month	day	dayofweek	hour	minute
        if (i == "month") | (i == "day") | (i == "dayofweek") | (i == "hour") | (i == "minute") | (i == "Open") | (i == "High") | (i == "Low"):
            continue
        data[i] = mms.fit_transform(data[[i]])
        if save: pickle.dump(mms, open(f"{dir}/{i}_scaler.pkl", 'wb'))

    return data

# rescale function to load rescalers from folder and apply them to corresponding columns
def rescale_load(data,modelName):
    """
    Rescale data using existing scalers from model folder
    """
    scalers = {}
    for i in data.columns:
        if (i == "month") | (i == "day") | (i == "dayofweek") | (i == "hour") | (i == "minute") | (i == "Open") | (i == "High") | (i == "Low"):
            continue

        mms = pickle.load(open(f'../models/{modelName}/scalers/{i}_scaler.pkl', 'rb'))
        scalers[f"{i}_scaler"] = mms
        data[i] = mms.transform(data[[i]])

    return scalers

def descaleFeature(data,modelName,featureName):
    """
    Descale given feature
    """
    mms = pickle.load(open(f'models/{modelName}/scalers/{featureName}_scaler.pkl', 'rb'))
    descaledTarget = (mms.inverse_transform(data[[featureName]]))
    
    return pd.DataFrame(descaledTarget).iloc[:,0].rename(f"descaled_{featureName}")


## 5.2 Nature transformation

Give cyclical nature to cyclical features

In [12]:

def cyclicalTime(data):
    """
    
    """
    data['month_sin'] = data['month'].apply(lambda x: np.sin(x*(2.*np.pi/12)))
    data['month_cos'] = data['month'].apply(lambda x: np.cos(x*(2.*np.pi/12)))

    data['day_of_month_sin'] = data['day'].apply(lambda x: np.sin(x*(2.*np.pi/30)))
    data['day_of_month_cos'] = data['day'].apply(lambda x: np.cos(x*(2.*np.pi/30)))

    data['dayofweek_cos'] = data['dayofweek'].apply(lambda x: np.cos(x*(2.*np.pi/7)))
    data['dayofweek_sin'] = data['dayofweek'].apply(lambda x: np.sin(x*(2.*np.pi/7)))

    data['hour_sin'] = data['hour'].apply(lambda x: np.sin(x*(2.*np.pi/24)))
    data['hour_cos'] = data['hour'].apply(lambda x: np.cos(x*(2.*np.pi/24)))

    # drop month, day, dayofweek, hour and minute columns from data
    data = data.drop(['month','day','dayofweek','hour'], axis=1)

    return data


## 5.3 Apply functions

Finally, apply all previously defined functions to data

In [13]:

print("generating target")
data = genTarget(data=data, rollingMeanWindow=rollingMeanWindow, predictionHorizon=predictionHorizon)

print("generating date features")
data = genDate(data)

print("processing open, high, low")
data = processOHL(data)

print("generating technical indicators")
data = genTechnicalIndicators(data,timePeriods)

print("rescaling")
data = rescale_gen(data,modelName=modelName,save=True)

print("generating cyclical features")
data = cyclicalTime(data)


generating target
generating date features
processing open, high, low
generating technical indicators


C:\Users\DONSA\AppData\Local\Temp\ipykernel_1344\3800710115.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["OBV"] = talib.OBV(data["Close"],data["Volume"])


# 6. Model training

## 6.1 Error functions

Define error functions to be used later

In [16]:

def mean_percentage_error(y, yhat):
    return np.mean( (y-yhat)/y)
     
def mean_absolute_percentage_error(y, yhat):
    return np.mean( np.abs((y-yhat)/y))

def smape_error(y, yhat):
  
    # Convert actual and predicted to numpy
    # array data type if not already
    if not all([isinstance(y, np.ndarray),isinstance(yhat, np.ndarray)]):
        y, yhat = np.array(y),np.array(yhat)
  
    return round(np.mean(np.abs(yhat - y) / ((np.abs(yhat) + np.abs(y))/2))*100, 4)

def rmse_error(y, yhat):
    return np.sqrt( mean_squared_error(y, yhat))

def ml_error(model_name, y, yhat):
    # when y or yhat is negative, give them 0 value
    y = np.where(y < 0, 0, y)
    yhat = np.where(yhat < 0, 0, yhat)

    mae = mean_absolute_error(y, yhat)
    smape = smape_error(y, yhat)
    mape = mean_absolute_percentage_error(y, yhat)
    rmse = rmse_error(y, yhat)
    msle = mean_squared_log_error(y, yhat, squared=True)
    rmsle = mean_squared_log_error(y, yhat, squared=False)
    
    return pd.DataFrame( { 'Model Name': model_name, 
                           'MAE': mae,
                           'SMAPE': smape,
                           'MAPE': mape,
                           'RMSE': rmse,
                           "MSLE" : msle,
                           'RMSLE': rmsle
                           },index=[0])


## 6.2 Split data

Finally split the data using function from 2

In [17]:

Train, Val, Test = splitDataRandom(TrainSize=0.84,ValSize=0.08,TestSize=0.08,data=data)

x_train = Train.drop(["Target"],axis=1)
y_train = Train["Target"]

x_val = Val.drop(["Target"],axis=1)
y_val = Val["Target"]

x_test = Test.drop(["Target"],axis=1)
y_test = Test["Target"]


## 6.3 Hyperparameter finetuning

skopt.gp_minize to speed up the process

In [20]:

def train_model(params):
    learning_rate = params[0]
    subsample = params[1]
    colsample_bytree = params[2]
    max_depth = params[3]
    print(params)
    model_xgb = xgb.XGBRegressor(
        eta=learning_rate,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        max_depth =max_depth,
        tree_method='gpu_hist',
        predictor="gpu_predictor",
    )
    model_xgb.fit(x_train, y_train)
    yhat = model_xgb.predict(x_val)
    error = rmse_error(y_val, yhat)
    print(f"ERROR: {error}")
    return error

space = [
        (.001, .6, 'log-uniform'), #learning rate
        (0.2, 1.0),    # subsample         
        (0.1, 1.0),     # colsample bytree  
        (5, 10)         # max_depth         
        ]


In [21]:

results_gp = skopt.gp_minimize(train_model, space, random_state=42, verbose=1, n_calls=50, n_random_starts=10)


Iteration No: 1 started. Evaluating function at random point.
[0.16327394826942576, 0.34674783189293107, 0.8017219002454925, 8]
ERROR: 0.016465814577023372
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 3.1509
Function value obtained: 0.0165
Current minimum: 0.0165
Iteration No: 2 started. Evaluating function at random point.
[0.017321712250831938, 0.27997993265440235, 0.5133240027692806, 7]
ERROR: 0.060356839616614186
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 2.1929
Function value obtained: 0.0604
Current minimum: 0.0165
Iteration No: 3 started. Evaluating function at random point.
[0.002494052716368271, 0.7207107783590825, 0.15077042112439026, 9]
ERROR: 0.12759322054114738
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 3.0659
Function value obtained: 0.1276
Current minimum: 0.0165
Iteration No: 4 started. Evaluating function at random point.
[0.40498727922258426, 0.20062301267281146, 0.992990403362096, 8]
ERROR: 0.03

Selected hyperparameters:

In [22]:

params = results_gp.x

params


[0.12314972219236571, 0.854402527268981, 0.1, 10]

## 6.4 Model evaluation

Instantiate a couple different models to compare

In [23]:

learning_rate = params[0]
subsample = params[1]
colsample_bytree = params[2]
max_depth = params[3]

model_xgb_bayes = xgb.XGBRegressor(
        eta=learning_rate,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        max_depth =max_depth,
        tree_method='gpu_hist',
        predictor="gpu_predictor",
    )
model_xgb_default = xgb.XGBRegressor(
        tree_method='gpu_hist',
        predictor="gpu_predictor",
    )
model_linear_regression = LinearRegression()


In [24]:

def computeModelErrors(model, resultsName, x_train, y_train, x_test, y_test):
    modelFit = model.fit(x_train, y_train)
    yhat = modelFit.predict(x_test)
    return ml_error(resultsName, y_test, yhat)


In [25]:

xgboost_bayes_result = computeModelErrors(model_xgb_bayes, "XGBoost Bayes", x_train, y_train, x_test, y_test)


In [26]:

xgboost_default_result = computeModelErrors(model_xgb_default, "XGBoost Default", x_train, y_train, x_test, y_test)


In [27]:

linear_result = computeModelErrors(model_linear_regression, "Linear", x_train, y_train, x_test, y_test)


In [28]:

pd.concat([
    xgboost_bayes_result,
    xgboost_default_result,
    linear_result,
    ])


,Model Name,MAE,SMAPE,MAPE,RMSE,MSLE,RMSLE
0,XGBoost Bayes,0.009013,1.6423,0.016917,0.012800,0.000065,0.008089
0,XGBoost Default,0.012705,2.2758,0.023841,0.017601,0.000124,0.011134
0,Linear,0.053690,9.3306,0.113141,0.072335,0.002070,0.045499


## 6.5 Save selected model

Finally, save model to folder for use in trading

In [29]:

x_all = data.drop(["Target"],axis=1)
y_all = data["Target"]


In [30]:

def saveModel(model, modelName, modelParamsDict):
    with open( f"../models/{modelName}/{modelName}.pkl", 'wb' ) as file:
        pickle.dump( model, file )
    # save model params dict to same folder
    with open( f"../models/{modelName}/{modelName}_params.json", 'w' ) as file:
        json.dump( modelParamsDict, file )


In [31]:

selectedModel = model_xgb_bayes.fit(x_all, y_all)
saveModel(selectedModel, modelName, modelParamsDict)
